In [3]:
import pandas as pd
import numpy as np
from sklearn.neighbors import LocalOutlierFactor

In [4]:
labeled_df = pd.read_csv('labeled_df.csv')
labeled_df.shape

(90960, 16)

In [5]:
labeled_df = labeled_df.drop(columns='Unnamed: 0')
labeled_df.head()

,customerId,timestamp,country,amount,businessName,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType
0,93000801,2021-09-15 16:13:35.147,6,150.00,c7543b6f04850eae288aa52783b33363,1,4,5,17,699,2013,144,19,113,5
1,93000801,2021-08-24 10:34:24.487,5,3200.00,c7543b6f04850eae288aa52783b33363,1,4,5,17,699,2013,144,19,113,5
2,93000801,2021-09-14 16:21:43.33,6,8350.00,c7543b6f04850eae288aa52783b33363,1,4,5,17,699,2013,144,19,113,5
3,93001001,2021-06-30 09:07:16.733,6,500.00,3f241dfcc38064261fec0f303c2784e5,1,0,5,17,123,1781,90,8,99,10
4,93001001,2021-07-12 16:36:02.857,6,1895.68,a5d1eaffa8e325d370665f43624339a1,1,0,5,17,285,1781,90,8,99,10


In [6]:
from sklearn.ensemble import IsolationForest
X=labeled_df.copy()
X=X.drop(columns=['businessName','customerId','timestamp'])
clf = LocalOutlierFactor(n_neighbors=20)
clf.fit_predict(X)

array([1, 1, 1, ..., 1, 1, 1])

In [8]:
encoder_dict=pickle.load(open('enc_dict.pkl','rb'))
columns_to_label = ['country', 'userAgent_browserFamily', 'userAgent_osFamily', 'IBAN_countryCode', 'IBAN_bankCode','CAP', 'SIA', 'SAE', 'RAE','companyType']

In [9]:
def inverse_and_scores(if_df,labeled_df,enc_dict,columns,model):
    df_copy=labeled_df.copy()
    label=model.fit_predict(X)
    scores=model.negative_outlier_factor_

    for clmn in columns:
        df_copy[clmn]=enc_dict[clmn].inverse_transform(df_copy[clmn])

    df_copy['scores'] = scores
    df_copy['label'] = label

    return df_copy

df_scores=inverse_and_scores(X,labeled_df,encoder_dict,columns_to_label,clf)

df_scores.head()

,customerId,timestamp,country,amount,businessName,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,scores,label
0,93000801,2021-09-15 16:13:35.147,Italy,150.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,-1.008318,1
1,93000801,2021-08-24 10:34:24.487,Greece,3200.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,-1.047175,1
2,93000801,2021-09-14 16:21:43.33,Italy,8350.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,-1.019783,1
3,93001001,2021-06-30 09:07:16.733,Italy,500.00,3f241dfcc38064261fec0f303c2784e5,1,Chrome,Windows,IT,2008,CEFGV,31039,430,505,SRL,-1.090301,1
4,93001001,2021-07-12 16:36:02.857,Italy,1895.68,a5d1eaffa8e325d370665f43624339a1,1,Chrome,Windows,IT,3069,CEFGV,31039,430,505,SRL,-1.006947,1


In [1]:
df_scores[df_scores['label']==-1].sort_values('scores')

NameError: name 'df_scores' is not defined